# WiDS DATATHON WORKSHOP
## FEBRUARY 15, 2020
<img src="images/inst_logos.png" alt="Harvard IACS" style="height: 80px;" align="left"/>

In [3]:
# Includes the necessary libraries
import numpy as np
import matplotlib.pylab as plt 
import os

from keras import backend as K
from keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import Callback, ModelCheckpoint, History 
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD, Adam

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, fetch_olivetti_faces
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, scale
from sklearn.metrics import confusion_matrix, roc_auc_score

%matplotlib inline

random_state = 0
random = np.random.RandomState(random_state)

# I. Classification with Non-linear Decision Boundaries



## Classification with Linear Boundaries
In **logistic regression**, we model the probability of an input $\mathbf{x}$ being labeled '1' as a function of its distance from the hyperplane parametrized by $\mathbf{w}$

<img src="./images/fig0.png" style='height:300px;'>

That is, we model $p(y=1 | \mathbf{w}, \mathbf{x}) = \sigma(\mathbf{w}^\top \mathbf{x})$. Where $\mathbf{w}^\top \mathbf{x}=0$ is the equation of the decision boundary.

## How would you parametrize a ellipitical decision boundary?

<img src="./images/fig1.png" style='height:300px;'>

We can say that the decision boundary is given by a ***quadratic function*** of the input:
$$
w_1x^2_1 + w_2x^2_2 + w_3 = 0
$$
We say that we can fit such a decision boundary using logistic regression with degree 2 polynomial features

## How would you parametrize a complex decision boundary?

<img src="./images/fig2.png" style='height:300px;'>

It's not easy to think of a function $g(x)$ can capture this decision boundary.

**GOAL:** Find models that can capture *arbitrarily complex* decision boundaries.

## The Decision Boundaries of Different Classifiers

We've proposed three types of models that can capture complex decision boundaries: 
1. logistic regression with polynomial boundaries
2. decision trees
3. KNN classifiers

<img src="./images/fig3.png" style='height:300px;'>

## An Embarassment of Riches

Now we have seen three types of classifiers: logistic regression, decision tree, kNN. Each type can be customized in many ways (e.g. we can choose many different polynomials for the logistic regression boundary)

***Question:*** which model should we use?

***Answer:*** your choice of model should depend on the task and the dataset. You must
1. choose models based on sensible evaluation metrics
2. choose models using proper data set splitting procedure (train/validation/test)
3. choose models that best solves your real-life task!

## Comparison of Classifiers: Computational Concerns
In addition to considering how well a model solves our real-life task, we also need to consider how easily we can apply a model to large amounts of data - this is called ***scalability***. 

Specifically, we want to know how easily we can train a model and how easily we can compute a prediction.

**QUESTION:** What are the advantages and disadvantages of each type of classifier with respect to scalability?

## How would you parametrize an arbitrary complex decision boundary?

<img src="./images/fig2.png" style='height:300px;'>

It's not easy to think of a function $g(x)$ can capture this decision boundary.

**REVISED GOAL:** Find models that can capture *arbitrarily complex* decision boundaries **and** are fast to train as well as effecient for computing predictions.

# Neural Networks

## Approximating Arbitrarily Complex Decision Boundaries

Given an exact parametrization, we could learn the functional form, $g$, of the decision boundary directly. 

However, assuming an exact form for $g$ is restrictive. 

Rather, we can build increasingly good approximations, $\widehat{g}$, of $g$ by composing simple functions. 

## What is a Neural Network?

**Goal:** build a good approximation $\widehat{g}$ of a complex function $g$ by composing simple functions.

For example, let the following picture represents $f\left(\sum_{i}w_ix_i\right)$, where $f$ is a non-linear transform:

<img src="./images/fig4.png" style='height:200px;'>

## Neural Networks as Function Approximators

Then we can define the approximation $\widehat{g}$ with a graphical schema representing a complex series of compositions and sums of the form, $f\left(\sum_{i}w_ix_i\right)$

<img src="./images/fig5.png" style='height:300px;'>

This is a ***neural network***. We denote the weights of the neural network collectively by $\mathbf{W}$.
The non-linear function $f$ is called the ***activation function***.

## Exercise: Translate Graphical Representations into Functional Ones

Translate the following graphical representation of a neural network into a functional form, $g(x_1, x_2) = ?$ 
<img src="./images/fig7.png" style='height:100px;'>
Use the following labels:
1. the input nodes $x_1$ and $x_2$
2. the hidden nodes $h_1$ and $h_2$
3. the output node $y$
4. the weight from $x_i$ to $h_j$ as $w^i_j$
5. the weight from $h_j$ to $y$ as $w^j$
6. the activation function $g(x) = e^{-0.5x^2}$

**Bonus:** write the functional form in vector notation.

## A Flexible Framework for Function Approximation


<img src="./images/fig6.png" style='height:500px;'>




## Common Choices for the Activation Function

<img src="./images/fig8.png" style='height:500px;'>

# Neural Networks for Classification

## Neural Network for Classification


**Data:** features `x_train`, real-valued labels `y_train`

**Probabilistic Model:** `y_train` $\sim \text{Bernoulli}(\sigma(g_\mathbf{W}($ `x_train` $))$, where $g_\mathbf{W}$ is a neural network with parameters $\mathbf{W}$, and $\sigma(z) = \frac{1}{1+e^{-z}}$.

**Training Objective:** find $\mathbf{W}$ to maximize the likelihood of our data. This is equivalent to minimizing the ***binary cross entropy*** or ***log loss***,
$$
\max_{\mathbf{W}}\, \mathrm{CrossEnt}(\mathbf{W}) =\sum^N_{n=1} y_n \log( g_\mathbf{W}(x_n)) + (1 - y_n) \log\left(1 - g_\mathbf{W}(x_n)\right)
$$

**Optimizing the Training Objective:** Typically when we optimize an objective function $\mathcal{L}$, we comput the gradient of the objective function with respective to the model parameters $\mathbf{W}$, set it equal to zero and solved for the optimal $\mathbf{W}$ analytically. 

Can we analytically optimize $\mathcal{L}$ when $g_\mathbf{W}$ is a neural network?

## Exercise: Optimizing Neural Networks

For the small neural network in the previous exercise, compute the gradient $\nabla_{\mathbf{W}}\,\mathrm{MSE}(\mathbf{W})$. Can you analytically solve for the optimal parameters $\mathbf{W}$? Is the training objective convex?

## Gradient Descent for Training Neural Networks: BackProp
The intuition behind various flavours of gradient descent  is as follows:

<img src="./images/fig9.pdf" style='height:400px;'>

## Gradient Descent: the Algorithm
1. start at random place: $W_0\leftarrow \textbf{random}$

2. until (stopping condition satisfied):

  a. compute gradient: 
     gradient = $\nabla$ loss\_function($W_{t}$)

  b. take a step in the negative gradient direction: 
     $W_{t+1} \leftarrow W_{t}$ - eta * gradient

Here *eta* is called the ***learning rate***.

# Implementing Neural Networks in `python`

## `keras`: a Python Library for Neural Networks

`keras` is a `python` library that provides intuitive api's for build neural networks quickly. 

``` python
#keras model for feedforward neural networks
from keras.models import Sequential

#keras model for layers in feedforward networks
from keras.layers import Dense

#keras model for optimizing training objectives
from keras import optimizers
```

## Building a Neural Network for Classification in `keras`

``` python
#instantiate a feedforward model
model = Sequential()

#add layers sequentially

#input layer: 2 input dimensions
model.add(Dense(2, input_dim=2, activation='relu', 
                kernel_initializer='random_uniform',
                bias_initializer='zeros')) 
#hidden layer: 2 nodes
model.add(Dense(2, activation='relu', 
                kernel_initializer='random_uniform',
                bias_initializer='zeros')) 

#output layer: 1 output dimension
model.add(Dense(1, activation='sigmoid', 
                kernel_initializer='random_uniform',
                bias_initializer='zeros')) 

#configure the model: specify training objective and training algorithm
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='binary_crossentropy')
```

## Training a Neural Network in `keras`

``` python
#fit the model and return the mean squared error during training
history = model.fit(X_train, Y_train, batch_size=20, shuffle=True, epochs=100, verbose=0)
```

## Training Your NN: Optimization Choices Matter
<img src="./images/fig10.jpg" style='height:400px;'>

## Monitoring Neural Network Training

Visualize the mean square error over the training, this is called the training ***trajectory***.

``` python
#fit the model and return the mean squared error during training
history = model.fit(X_train, Y_train, batch_size=20, shuffle=True, epochs=100, verbose=0)

# Plot the loss function and the evaluation metric over the course of training
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.plot(np.array(history.history['mean_squared_error']), color='blue', label='training accuracy')

plt.show()
```

## Diagnosing Issues with the Trajectory
If this is your objective function during training, what can you conclude about your step-size?
<img src="./images/fig13.png" style='height:400px;'>

## Diagnosing Issues with the Trajectory
If this is your objective function during training, what can you conclude about your step-size?
<img src="./images/fig14.png" style='height:400px;'>

## Diagnosing Issues with the Trajectory
If this is your objective function during training, what can you conclude about your step-size?
<img src="./images/fig15.png" style='height:400px;'>

## Exercise: Train a Neural Network Classifier

In the `WiDS_Starter_Notebook_3.ipynb` `colab` notebook, Part II: investigate the effectiveness of a neural network classifier on a toy dataset. 

Why is the neural network so effective on this dataset?

## What Does a Neural Network Learn?

<table>
    <tr><td><font size="3">A Complex Decision Boundary $g\quad\quad\quad\quad\quad$</font></td>
        <td><font size="3">A Transformation $g_0$ and a linear model $g_1\quad\quad\quad\quad$</font></td>
    <tr><td><img src="images/decision.png" style="height:350px;"></td>
        <td><img src="images/architecture2.png" style="height:400px;"></td></tr>
</table>

# Class Imbalance

## Data stratification

When the classes in your data set are extremely unbalanced, a random split of your data into training, validation and test may result in one of these sets containing **zero** instances of the rare class.

**Problem:** Training on only data containing no rare labels will result in a classifier that is unable to generalize and predict well on data with these labels that may be present in test set.


***Stratification*** is the technique to allocate the samples evenly based on sample classes so that training set and validation set have similar ratio of classes.

## Reweighting classes
When classes in your data set are extremely unbalanced, the models you train can be unincentivized to predict correctly on the rare class -- specializing on the overrepresented classes will result in low average loss. 

We can ***reweight*** the terms in the loss function so that contributions from terms associated with the rare class is increased while the contribution of those associated with the overrepresented classes is decreased. Intuitively, the model is penalized more for being 'wrong' on the rare class.

<img src="./images/unbalanced.png" style="width: 500px;" align="center"/>

## Resampling points

We can also create a subset of the data that has a more balanced representation of all classes by: 

1. Down-sampling (Under sampling) the majority class
2. Up-sampling (Over sampling) the minority class
3. Fanciers sampling techniques

<img src="./images/sampling.png" style="width: 500px;" align="center"/>


# The Bias Variance Trade-off

## Deep vs Shallow Trees: The Bias Variance Trade Off
If you randomly draw 50 sample from our toy data with non-linear boundary and fit a decision tree on each sample individually, their decision boundaries will look like the following:

<img src="./images/tree_boundaries.png" alt="" style="height: 300px;"/>

What does this say about the variance of very deep trees? How does this impact the generalization of deep trees?

## Overfitting
Complex models have ***low bias*** -- they can model a wide range of functions, given enough samples.

But complex models can use their 'extra' capacity to explain non-meaningful features of the training data that are unlikely to appear in the test data (i.e. noise). These models have ***high variance*** -- they are very sensitive to small changes in the data distribution, leading to drastic performance decrease from train to test settings.

<img src="./images/tree_boundary.png" style="width: 500px;" align="center"/>


## Regularization
A way to prevent overfitting is to reduce the capacity of the model, thereby limiting the kinds of functions they can model. This **increases bias, but reduces variance**:
1. **$\ell_1$, $\ell_2$ weight regularization** - adding a term to the loss function that penalizes the $\ell_1$-norm (sum of absolute values) or the $\ell_2$-norm (sum of squares) of the weights. This prevents the network from learning extremely squiggly functions.
``` python
from keras import regularizers
model.add(Dense(64, input_dim=64,
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
```

2. **Dropout** - randomly zeroing out weights during training. This prevents the hidden nodes from "over specializing" or "memorizing" certain data points.
``` python
from keras.layers import Dropout,
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
```


## Bagging
Another way to prevent overfitting is to create ***ensembles*** of models. ***Bagging*** (Bootstrapping and Aggregating) is an ensemble method that:
1. trains a diverse set of complex models on various subsets of the training data -- low bias but high variance
2. reduces the variance by averaging the outputs of these models -- random mistakes will 'cancel out' in the average


<img src="./images/bagging.jpg" style="width: 300px;" align="center"/>

## Random Forest Implementation in `sklearn`

A **random forest** is the 'averaged model' of collection of deep decision trees each learned on a subset of the training data (each branch in each tree is also trained on a randomized set of input dimensions to reduce correlation between trees).

``` python
#import the random forest model
from sklearn.ensemble import RandomForestClassifier

#instantiate a forest of with 1000 trees, each of depth 1000
forest = RandomForestClassifier(n_estimators=1000, max_depth=1000)

#fit your forest to data
forest.fit(x_train, y_train)

#predict new labels using the fitted forest
forest.predict(x_test)
```

## For Practitioners:

A random forest, as implemented in `sklearn`, has a number of hyper-parameters that effects model performance. Some important hyper-parameters are:

1. number of trees in the forest `n_estimators`
2. max depth of trees `max_depth`
3. hyper-parameters that determines the subsets of input dimensions to learn each branch and the subsets of training data to learn each tree

## Boosting

***Boosting*** is an ensemble method that builds a complex model iteratively by summing together simple models:
1. train a simple model -- high bias but low variance
2. train a simple model to focus on the *errors* of the current model, add the new model to the existing model
3. repeat

***Gradient boosting*** is easy to implement by hand.

***Adaboost*** (Adaptive boosting) is implemented by `sklearn`, and can be made to work with neural network models implemented in `keras`